<a href="https://colab.research.google.com/github/lilymarzano/LMSpatialDataFrames/blob/master/Neural_Network_Tutorial_and_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Neural Network Regression and Classification**

---

## **Overview**
This tutorial provides a step-by-step guide to building and training **neural networks** for both **regression** and **classification** tasks.  
You will learn how to implement models using **Scikit-learn** and **TensorFlow/Keras**, perform **hyperparameter tuning**, and interpret model performance using key evaluation metrics.

---

## **Learning Objectives**
By the end of this tutorial, you will be able to:

1. Implement neural networks for **regression** and **classification** tasks using Scikit-learn and TensorFlow.  
2. Apply **hyperparameter optimization** (e.g., Random Search, Grid Search, or KerasTuner) to systematically improve model performance.  
3. Select and justify appropriate **activation functions**, **architectures**, and **training configurations** for different machine learning problems.  
4. Evaluate model performance using key metrics such as **RMSE**, **R²**, and **classification accuracy**.

---

## **Tutorial Outline**
This notebook is divided into two main parts:

### **Part 1 — Regression Task: Predicting Crop Yield**
- You will use an environmental dataset to predict **USDA county-level crop yields**.  
- In *Assignment 3*, you trained tree-based models (e.g., Random Forest and XGBoost).  
- Here, you will extend that work by building a **neural network regressor**, capable of modeling complex, nonlinear relationships between climate variables and crop productivity.

### **Part 2 — Classification Task: Recognizing Handwritten Digits**
- You will explore the **MNIST dataset**, which contains images of handwritten digits (0–9).  
- This section demonstrates how neural networks perform **image recognition** and **multi-class classification** using **softmax activation** for probabilistic prediction.

---

## **Why Learn Both Tasks?**
- **Regression** shows how neural networks predict **continuous outcomes** (e.g., yield).  
- **Classification** illustrates how they categorize **discrete classes** (e.g., digits).  
Mastering both prepares you to design neural architectures suited to a wide range of real-world applications — from climate-based yield forecasting to computer vision.

---


# **0. Import Necessary Packages**

Before we begin, we need to import all the essential Python libraries used in this tutorial.  
Importing packages at the top of your notebook ensures **clarity**, **reproducibility**, and **ease of debugging**.

**Installation Notes**

Before running this tutorial, ensure all required libraries are installed.  
💡 Run these commands **only once per environment** (e.g., in Jupyter Notebook, VS Code, or a Conda environment).

```bash
# Install all core packages for this tutorial
%pip install tensorflow keras_tuner

# (Optional) Upgrade TensorFlow to the latest version if needed
%pip install --upgrade --force-reinstall tensorflow


In [ ]:
%pip install tensorflow
%pip install --upgrade --force-reinstall tensorflow
%pip install keras_tuner

  Using cached tensorflow-2.20.0-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (4.5 kB)
  Using cached absl_py-2.3.1-py3-none-any.whl.metadata (3.3 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl.metadata (4.4 kB)
  Using cached flatbuffers-25.9.23-py2.py3-none-any.whl.metadata (875 bytes)
  Using cached gast-0.6.0-py3-none-any.whl.metadata (1.3 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl.metadata (814 bytes)
  Using cached libclang-18.1.1-py2.py3-none-manylinux2010_x86_64.whl.metadata (5.2 kB)
  Using cached opt_einsum-3.4.0-py3-none-any.whl.metadata (6.3 kB)
  Using cached packaging-25.0-py3-none-any.whl.metadata (3.3 kB)
  Using cached protobuf-6.33.1-cp39-abi3-manylinux2014_x86_64.whl.metadata (593 bytes)
  Using cached requests-2.32.5-py3-none-any.whl.metadata (4.9 kB)
  Using cached setuptools-80.9.0-py3-none-any.whl.metadata (6.6 kB)
  Using cached six-1.17.0-py2.py3-none-any.whl.metadata (1.7 kB)
  Using cached termcolor-3.2.0-py3-none-

In [ ]:
# ============================================================
# Core Python Libraries
# ============================================================
import os
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# ============================================================
# Scikit-learn (Machine Learning Utilities)
# ============================================================
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error, r2_score

# ============================================================
# TensorFlow / Keras (Deep Learning Framework)
# ============================================================
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Sequential, layers, models
from tensorflow.keras.layers import Dense, Normalization, Dropout, BatchNormalization
from tensorflow.keras.metrics import RootMeanSquaredError
from tensorflow.keras.callbacks import EarlyStopping

# ============================================================
# Keras Tuner (Hyperparameter Optimization)
# ============================================================
import keras_tuner as kt

# ============================================================
# Spatial Tools (for Mapping and Visualization)
# ============================================================
import geopandas as gpd

# ============================================================
# Warning Control & Random Seed (for Reproducibility)
# ============================================================
from sklearn.exceptions import ConvergenceWarning

# Suppress non-critical warnings for cleaner output
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=RuntimeWarning)

# Set random seeds for reproducibility
SEED = 42
tf.random.set_seed(SEED)

print(f"Libraries successfully loaded. Random seed set to {SEED}. TensorFlow version: {tf.__version__}")

# **Part 1 — Regression: Predicting Crop Yield**

This section demonstrates how to build and evaluate a **neural network regression model** to predict **USDA county-level crop yields** using climate and environmental features.  
You will begin by exploring and preparing the dataset, defining the features and target variable, and finally splitting the data into training and testing subsets.

---

## **1.1 Load the Data and Perform a Quick Data Check**

💡 **Tips**

Loading your dataset correctly is the first step toward a clean and reproducible analysis.

**Best Practices:**
- Use `pd.read_csv()` to load tabular data.  
- Specify **data types** for key identifier columns (e.g., `GEOID`, `STATEFP`) as `str` to preserve **leading zeros**, which are important for spatial joins and FIPS mapping.  
- Perform quick data inspections to verify correct loading and structure.

**Recommended checks:**
- `df.shape` → shows the number of rows (samples) and columns (features + target).  
- `df.head()` → previews the first few rows to confirm column names and formats.  
- For large datasets, load only a few rows first using `nrows=5` to preview structure before reading the full file.

In [ ]:
# Load dataset
df = pd.read_csv("US_corn.csv", dtype={"GEOID": str, "STATEFP": str})

# Quick checks
print("Data shape:", df.shape)
df.head()

## **1.2 Define Features (X) and Target (y)**

💡 **Tips:**  
- In supervised machine learning, we separate the dataset into:  
  - **Predictors (features)** → the input variables the model will use.  
  - **Target (label)** → the variable we want to predict.  
- Here:  
  - The variable **`ln_yield`** is the **target** representing the natural log of crop yield (stabilizes variance and improves regression performance).  
  - The selected **features** include climate indicators that influence yield outcomes.  
- Define:  
  - `X = df[features]` → extracts only the chosen feature columns from the full DataFrame.  
  - `y = df["ln_yield"]` → sets the target variable for prediction.  
- Handle categorical variables:  
  - `pd.get_dummies(X, columns=['STATEFP'])` converts the **state code** (`STATEFP`) from categorical to multiple binary (0/1) columns — one per state.  
  - This process, known as **one-hot encoding**, lets models like Random Forest or XGBoost interpret categorical information numerically.  
  - For tree-based models, keep all dummy columns; for linear models, use `drop_first=True` to avoid multicollinearity.  
- Data validation checks:  
  - `X.shape` and `y.shape` → confirm that the number of rows matches between predictors and target.  
  - `X.isna().any().any()` and `y.isna().any()` → test for missing values in features and target.  
  - If missing values exist:  
    - For **features (`X`)**, impute using mean or median.  
    - For **target (`y`)**, drop rows with missing labels to avoid training errors.  

In [ ]:
# Define feature set and target variable
features = [
    'STATEFP',
    'sm',
    'precip',
    'tmean',
    'vpdmean'
]

# Select features (X) and target (y)
X = df[features]
y = df["ln_yield"]

# Encode categorical variables
X = pd.get_dummies(X, columns=['STATEFP'])

# Quick data quality checks
print("X shape:", X.shape)
print("y shape:", y.shape)
print("Any NA in X:", X.isna().any().any())
print("Any NA in y:", y.isna().any())

# If NA found in y, drop corresponding rows and then rerun X and y selection
# df = df.dropna(subset=['ln_yield'])


## **1.3 Prepare Training and Test Data (Train–Test Split)**

💡 **Tips:**  
- A **train–test split** divides the dataset into two parts to evaluate how well a model generalizes to unseen data:  
  - **Training set (70%)** → used by the model to learn patterns and relationships between features (`X`) and the target (`y`).  
  - **Test set (30%)** → held out and only used for the final evaluation of model performance.  
- The Scikit-Learn function `train_test_split()` automatically partitions your data:  
  - `test_size=0.30` → reserves 30% of the data for testing (you can adjust this depending on dataset size).  
  - `random_state=42` → sets a **random seed** to make the split **reproducible**, ensuring consistent results every time you run the code.  

In [ ]:
# Split data into training (70%) and testing (30%) sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42
)

# Display the resulting dataset shapes
print("Training set shape:", X_train.shape)
print("Testing set shape: ", X_test.shape)

## **1.4 Neural Network Modeling: Using Scikit-learn**

### **1.4.1 Baseline Neural Network**

Before tuning or building more complex networks, it’s essential to first create a **baseline model**.  
A baseline provides a benchmark against which later improvements can be measured.

In this section, we’ll use Scikit-learn’s `MLPRegressor` (Multilayer Perceptron) to fit a simple **feed-forward neural network** for crop yield prediction.

💡 **Tips**

- **Start simple:**  
  Begin with a small network to establish a baseline performance before tuning.  

- **Scale features:**  
  Neural networks are sensitive to feature magnitudes.  
  Use `StandardScaler()` to standardize features, which helps stabilize and accelerate convergence.  

- **Model architecture:**  
  A model with one hidden layer of 10 neurons (`hidden_layer_sizes=(10,)`) is a good minimal starting point.  

- **Use a Pipeline:**  
  Combine preprocessing and modeling steps using Scikit-learn’s `Pipeline`.  
  This approach prevents data leakage and ensures consistent transformations across training and prediction.  

- **Evaluate performance:**  
  Use both **RMSE (Root Mean Squared Error)** — to measure prediction error in target units — and **R² (Coefficient of Determination)** — to evaluate how well the model explains variance in the data.  

- **Convergence warnings:**  
  If you encounter convergence warnings, increase `max_iter` (e.g., to `1000`).  


In [ ]:
# Build a pipeline: scaling + simple MLP (1 hidden layer with 10 neurons)
nn_base = Pipeline([
    ("scaler", StandardScaler()),
    ("mlp", MLPRegressor(
        hidden_layer_sizes=(10,),  # one hidden layer with 10 neurons
        random_state=42,
        max_iter=1000              # increase if you see convergence warnings
    ))
])

# Fit the baseline model
nn_base.fit(X_train, y_train)

# Predict on the test set
y_pred = nn_base.predict(X_test)

# Evaluate model performance
rmse_base = np.sqrt(mean_squared_error(y_test, y_pred)) # Calculate RMSE by taking the square root of MSE
r2_base = r2_score(y_test, y_pred)

print("Baseline Evaluation:")
print(f"RMSE: {rmse_base:.4f}")
print(f"R²  : {r2_base:.4f}")

### **1.4.2 Define Hyperparameter Search Space**

💡 **Tips**

- **Why tune?**  
  Default parameters are rarely optimal — tuning helps balance **underfitting** (too simple) and **overfitting** (too complex).

- **Pipeline parameters:**  
  When tuning a model inside a `Pipeline`, prefix parameter names with the pipeline step, e.g., `"mlp__hidden_layer_sizes"`.

- **Common hyperparameters:**
  - `hidden_layer_sizes`: defines the number of neurons and layers (e.g., `(64, 32)` → 2 hidden layers).  
  - `activation`: nonlinear function that introduces flexibility (`'relu'`, `'tanh'`).  
  - `alpha`: L2 regularization term that penalizes large weights to prevent overfitting.  
  - `learning_rate_init`: initial step size for weight updates.  
  - `early_stopping`: halts training automatically if validation performance stops improving.  
  - `validation_fraction`: fraction of the training data reserved for validation when `early_stopping=True`.  
    - By default, 10% (`0.1`) of the training data is held out for validation.  
    - The model monitors validation loss and stops early when no improvement is seen for several epochs.  
    - This reduces overfitting and saves computation time.

In [ ]:
param_dist = {
    "mlp__hidden_layer_sizes": [(32,), (64,), (64, 32), (128, 64, 32)],
    "mlp__activation": ["relu", "tanh"],
    "mlp__alpha": [1e-4, 1e-3, 1e-2],        # L2 regularization strength
    "mlp__learning_rate_init": [1e-3, 1e-4],
    "mlp__early_stopping": [True],
    "mlp__validation_fraction": [0.1],
}

### **1.4.3 Randomized Search with Cross-Validation**

💡 **Tips**
- **Why `RandomizedSearchCV`?**  
  - Tests a *random subset* of parameter combinations instead of *every* combination (like Grid Search).  
  - Much faster for large search spaces and still finds strong models.  
- **Key arguments:**
  - `n_iter=5`: tries 5 random parameter sets.  
  - `cv=5`: uses **5-fold cross-validation** → trains on 4 folds, validates on 1, repeats 5× for more reliable performance.  
  - `scoring="neg_mean_squared_error"`: minimizes RMSE (Scikit-learn flips the sign because higher = better).  
  - `n_jobs=-1`: uses all CPU cores for faster training.  
  - `verbose=1`: prints progress to monitor the search.  
- **Outputs to check:**
  - `best_params_`: hyperparameters of the best model.  
  - `best_score_`: mean CV performance (take square root of negative value for RMSE).

In [ ]:
# Initialize randomized search
random_search = RandomizedSearchCV(
    estimator=nn_base,                   # pipeline defined earlier
    param_distributions=param_dist,
    n_iter=5,                           # number of random combinations to try
    scoring="neg_mean_squared_error",    # optimize for RMSE
    cv=5,
    verbose=1,
    random_state=42,
    n_jobs=-1
)

# Run search on training data
random_search.fit(X_train, y_train)

# Print results
best_params = random_search.best_params_
best_cv_rmse = np.sqrt(-random_search.best_score_)

print("Best Parameters:", best_params)
print(f"Best Cross-Validation RMSE: {best_cv_rmse:.4f}")


### **1.4.4 Refit Best Model and Evaluate on Test Set**

💡 **Tips**
- After hyperparameter tuning, the **best model** is stored in `random_search.best_estimator_`.  
- By default, Scikit-learn refits this best model on the **entire training set**, so it’s ready for prediction.  
- If you used `refit=False` in `RandomizedSearchCV`, call `.fit()` manually before prediction.  
- Evaluate model generalization on the **held-out test set** using:
  - **RMSE (Root Mean Squared Error)** → lower values mean better accuracy.  
  - **R² (Coefficient of Determination)** → closer to 1 indicates better variance explanation.  
- Always compare against your **baseline model**:
  - ↓ RMSE and ↑ R² → tuning improved performance.  
  - Small improvement → baseline was already strong.  
- **Visual check:**  
  - Plot *Observed vs. Predicted* — perfect predictions fall along the red dashed **1:1 line**.  
  - Patterns or scatter reveal bias and variance.


In [ ]:
# Get best model from RandomizedSearchCV
best_model = random_search.best_estimator_

# Predictions on the test set
y_pred_tuned = best_model.predict(X_test)

# Evaluate tuned model
rmse_tuned = np.sqrt(mean_squared_error(y_test, y_pred_tuned)) # Calculate RMSE by taking the square root of MSE
r2_tuned = r2_score(y_test, y_pred_tuned)

# Compare with baseline model (make sure you ran Section 4 first)
print(" Model Comparison (Test Set)")
print(f"Baseline NN → RMSE: {rmse_base:.4f},  R²: {r2_base:.4f}")
print(f"Tuned NN    → RMSE: {rmse_tuned:.4f},  R²: {r2_tuned:.4f}")

In [ ]:
# Optional: Visualize Observed vs Predicted
plt.figure(figsize=(5,4))
plt.scatter(y_test, y_pred_tuned, alpha=0.6, edgecolor='none')
lims = [min(y_test.min(), y_pred_tuned.min()), max(y_test.max(), y_pred_tuned.max())]
plt.plot(lims, lims, 'r--', linewidth=1.2, label="Perfect prediction (1:1 line)")
plt.xlabel("Observed ln(yield)", fontsize = 14)
plt.ylabel("Predicted ln(yield)", fontsize = 14)
plt.title("Observed vs Predicted (Tuned Neural Network)", fontsize = 14)
plt.legend()
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()


## **1.5 Neural Network Modeling: Using TensorFlow**

### **1.5.1 Baseline Neural Network**

In this section, you’ll build a **simple feed-forward neural network** using **TensorFlow/Keras** for regression.  
This model establishes a **baseline performance** that you can later improve with additional layers, regularization, or hyperparameter tuning.

💡 **Tips**

- **Start simple:**  
  Begin with one or two dense layers to create a straightforward baseline model.  
  Simplicity at this stage makes it easier to interpret performance gains after tuning.

- **Normalize inputs:**  
  Use Keras’s `Normalization()` layer to standardize feature values (mean = 0, variance = 1).  
  This improves numerical stability and convergence speed during training.

- **Build a Sequential model:**  
  Stack layers in order:  
  1. **Normalization layer** → standardizes inputs.  
  2. **Dense (hidden) layers** → learn nonlinear relationships between features.  
  3. **Dense (output) layer** → produces continuous yield predictions.

- **Compile the model:**  
  - **Optimizer:** `'adam'` — adaptive learning rate; a good default for most problems.  
  - **Loss:** `'mse'` — Mean Squared Error, standard for regression tasks.  
  - **Metrics:** `RootMeanSquaredError()` — interpretable in the same units as the target variable.

- **Train the model:**  
  Use `epochs=100` initially to give the optimizer enough time to converge.  
  Monitor the loss trend to determine if additional training or early stopping is needed.

- **Evaluate results:**  
  Compare predicted and actual yields using **RMSE** and **R²** to assess model accuracy and goodness of fit.

In [ ]:
# 1. Normalize input features (fit on training data only)
normalizer = Normalization()
normalizer.adapt(X_train.values)  # learns mean & variance from training features

# 2. Define a simple Sequential model
model = Sequential([
    normalizer,                           # normalization layer
    Dense(units=25, activation='relu'),   # hidden layer (25 neurons, ReLU activation)
    Dense(units=15, activation='sigmoid'),# second hidden layer (15 neurons, Sigmoid activation)
    Dense(units=1, activation='linear')   # output layer for regression
])

# 3. Compile the model
model.compile(
    optimizer='adam',                     # adaptive learning rate optimizer
    loss='mse',                           # mean squared error loss
    metrics=[RootMeanSquaredError()]      # RMSE for interpretability
)

# 4. Train the baseline model
history = model.fit(
    X_train, y_train,
    epochs=100,
    verbose=1                             # display training progress
)

# 5. Predict on the test set
y_pred = model.predict(X_test)

# 6. Evaluate baseline performance
rmse_base = np.sqrt(mean_squared_error(y_test, y_pred)) # Calculate RMSE by taking the square root of MSE
r2_base = r2_score(y_test, y_pred)

print("\nBaseline Neural Network Evaluation")
print(f"RMSE: {rmse_base:.4f}")
print(f"R²  : {r2_base:.4f}")

In [ ]:
## Model Summary and Interpretation
## After training the baseline TensorFlow/Keras model, we can examine its architecture using the command:

model.summary()

### **Model Summary Interpretation**

The table above summarizes the structure and parameter count of your **baseline TensorFlow neural network**.

| Component | Description | Interpretation |
|------------|--------------|----------------|
| **Model:** `"sequential"` | Indicates the model is a **Sequential** Keras model, meaning layers are stacked in order — each feeding into the next. | Simplifies design; ideal for straightforward feed-forward networks. |
| **`normalization_1 (Normalization)`** | Standardizes inputs using the mean and variance learned during `normalizer.adapt(X_train)`. | Ensures all features are on a similar scale, helping the network converge faster and more stably. |
| **`dense` (25 units, ReLU)** | Fully connected layer with 25 neurons using the ReLU activation function. | Learns nonlinear relationships among input features. |
| **`dense_1` (15 units, Sigmoid)** | Second hidden layer with 15 neurons using the sigmoid activation. | Captures finer nonlinearities and introduces a smooth, bounded activation for complex interactions. |
| **`dense_2` (1 unit, Linear)** | Output layer with one neuron and linear activation. | Produces continuous numeric output — appropriate for regression tasks. |

---

### **Parameter Breakdown**

- **Normalization layer:** `33` parameters  
  → These are *non-trainable statistics* (mean and variance) used to normalize input features.  
- **Dense(25):** `16 × 25 + 25 = 425` parameters  
  → Each neuron has 16 input weights (one per input feature) plus a bias term.  
- **Dense(15):** `25 × 15 + 15 = 390` parameters  
  → Connects all 25 outputs from the previous layer to 15 neurons, plus biases.  
- **Dense(1):** `15 × 1 + 1 = 16` parameters  
  → Maps 15 features to one final continuous prediction.  

---

### **Totals**
- **Total parameters:** 864  
- **Trainable parameters:** 831 (weights and biases learned during training).  
- **Non-trainable parameters:** 33 (mean and variance values from the Normalization layer).  

### **1.5.2 Early Stopping**

Training deep learning models for too many epochs can lead to **overfitting**, where the model performs well on training data but poorly on unseen data.  
To prevent this, we use **EarlyStopping**, a Keras callback that halts training when performance on a validation set stops improving.

---

💡 **Tips**

- **Purpose:** Early stopping monitors a chosen metric (e.g., validation loss) and **stops training automatically** when improvement stalls.  
- **Key arguments:**
  - **`monitor='val_loss'`** → tracks validation loss after each epoch.  
  - **`patience=5`** → waits for 5 epochs of no improvement before stopping (acts as a tolerance buffer).  
  - **`restore_best_weights=True`** → reloads the model weights from the epoch with the best validation loss (prevents using overfitted weights).  
  - **`verbose=1`** → prints a message when training stops early.  
- **Validation data:**
  - `validation_split=0.2` → automatically reserves 20% of the training data for validation, or  
  - A separate validation dataset (`X_val`, `y_val`).  Use `validation_split=0.2` (or provide a separate validation set) so the callback has data to monitor.  
- **Epochs setting:**
  - You can set a high number of epochs (e.g., 100–200). EarlyStopping will typically stop training much earlier once convergence is reached.  
  - This approach avoids manual tuning of epoch counts.
- **Batch Size:**
    The `batch_size` parameter controls how many training samples are processed before the model’s internal weights are updated.

    - **Typical values:** 16, 32, 64, or 128 — depending on dataset size and hardware.  
    - **Smaller batches:** More frequent updates → better generalization but slower training.  
    - **Larger batches:** Faster per epoch but may underfit or converge to suboptimal minima.  
    - **Recommendation:** Start with `batch_size=32` (a balanced default) and adjust based on model stability or training speed.


**Rationale**

Early stopping provides several benefits:
- **Prevents overfitting:** Stops once the validation error begins to increase.  
- **Saves time:** Avoids unnecessary training beyond the optimal point.  
- **Improves generalization:** Retains the best model observed during training.

In [ ]:
# 1. Define the EarlyStopping callback
earlystop = EarlyStopping(
    monitor='val_loss',          # metric to monitor
    patience=5,                  # epochs to wait before stopping
    restore_best_weights=True,   # revert to best model weights
    verbose=1                    # print when training stops early
)

# 2. Train the model with a validation split
history = model.fit(
    X_train, y_train,
    validation_split=0.2,        # 20% of training data used for validation
    epochs=100,                  # high limit; EarlyStopping will stop earlier
    batch_size=32,
    callbacks=[earlystop],
    verbose=1
)

# 3. Predict on test data
y_pred = model.predict(X_test)

# 4. Evaluate model performance
rmse_earlystop = np.sqrt(mean_squared_error(y_test, y_pred))
r2_earlystop = r2_score(y_test, y_pred)


# 5. Compare with baseline model without early stopping
print("\n Model Performance on Test Set (with Early Stopping):")
print(f"RMSE : {rmse_earlystop:.4f}")
print(f"R²   : {r2_earlystop:.4f}")

print("\n Comparison with Baseline Model:")
print(f"Baseline RMSE : {rmse_base:.4f}")
print(f"Baseline R²   : {r2_base:.4f}")

### **1.5.3 Building a Tunable Neural Network with KerasTuner**

💡 **Tips**

- **Core Function:**  
    Use **KerasTuner** to automatically explore and evaluate different **hyperparameter combinations** to find the best-performing neural network configuration that minimizes **validation error** (e.g., RMSE).  

    This model, built with the **Sequential API**, performs regression on normalized features while tuning key parameters such as:
    - Number of neurons per layer,  
    - Activation functions, and  
    - Learning rate.

- **Search Process:**  
    KerasTuner automates hyperparameter tuning by:
    1. Building, training, and evaluating multiple model configurations with different hyperparameter values.  
    2. Selecting the configuration that achieves the **lowest validation loss** (MSE or RMSE).  
    3. Reporting the optimal hyperparameters for retraining on the full training set.

- **Key Hyperparameters to Explore:**

| **Hyperparameter** | **Description** | **Effect** |
|--------------------|----------------|-------------|
| **`units`** | Number of neurons in each hidden layer (controls model capacity). | Too few → underfitting; Too many → overfitting or slower training. |
| **`activation`** | Nonlinear transformation applied to layer outputs (`'relu'`, `'tanh'`, `'sigmoid'`). | Enables the network to learn complex, nonlinear feature relationships. |
| **`learning_rate`** | Step size for optimizer updates (`1e-2`, `1e-3`, `1e-4`). | Smaller → stable convergence; Larger → faster training but may overshoot minima. |
| **`dropout`** *(optional)* | Fraction of neurons randomly dropped during training (0.0–0.4). | Reduces overfitting and improves generalization. |

- **Model Architecture (Sequential API):**
  1. **Normalization Layer:**  
     - Standardizes input features using a pre-adapted normalizer (`normalizer.adapt(X_train)` was executed earlier).  
     - Ensures that all features contribute evenly during training.  
  2. **Hidden Layers:**  
     - Two fully connected (`Dense`) layers with tunable `units` and `activation`.  
     - Capture complex nonlinear relationships between input features.  
  3. **Output Layer:**  
     - A single neuron with **linear activation**, suitable for continuous-valued regression outputs.  
  4. **Optimizer:**  
     - Uses the **Adam** optimizer with a tunable `learning_rate` to balance training stability and convergence speed.

- **Key Keras Functions:**
  - **`model = keras.Sequential()`** → Initializes a linear stack of layers where each layer feeds directly into the next.  
  - **`model.add(layer)`** → Appends a new layer to the model in sequential order.  
  - **`model.compile()`** → Configures the model for training by specifying:  
    - **`optimizer`** → The algorithm for updating weights (e.g., Adam).  
    - **`loss`** → The objective function to minimize (e.g., MSE for regression).  
    - **`metrics`** → Additional metrics to monitor (e.g., RMSE).

- **Outcome:**  
  - After tuning, KerasTuner reports the **best hyperparameter configuration** (e.g., number of units, activation function, and learning rate) that yields the **lowest validation RMSE**.  
  - You can then rebuild and retrain the model using these optimized settings to achieve the best regression performance.


In [ ]:
# The 'normalizer' has already been adapted on X_train
def build_model(hp):
    """
    KerasTuner-compatible model builder using the Sequential API.
    Each hidden layer can have its own 'units' and 'activation'.
    """
    model = keras.Sequential(name="regression_model")

    # Input normalization (use pre-adapted normalizer)
    model.add(normalizer)

    # ===== Layer 1 hyperparameters =====
    units_1 = hp.Int("units_1", min_value=16, max_value=128, step=16)
    activation_1 = hp.Choice("activation_1", ["relu", "tanh", "sigmoid"])

    # ===== Layer 2 hyperparameters (independent) =====
    units_2 = hp.Int("units_2", min_value=8, max_value=128, step=8)
    activation_2 = hp.Choice("activation_2", ["relu", "tanh", "sigmoid"])

    # Hidden layers (apply each to the previous tensor using Sequential.add)
    model.add(layers.Dense(units_1, activation=activation_1, name="hidden_1"))
    model.add(layers.Dense(units_2, activation=activation_2, name="hidden_2"))

    # Output layer for regression (single numeric value)
    model.add(layers.Dense(1, activation="linear", name="output"))

    # Tunable learning rate for Adam optimizer
    lr = hp.Choice("learning_rate", [1e-2, 1e-3, 1e-4])

    # Compile the model
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=lr),
        loss="mse",
        metrics=[RootMeanSquaredError(name="rmse")]
    )

    return model


### **1.5.4 Find the Best Hyperparameters Using Random Search**

💡 **Tips**

- **Purpose:**  
  - Automatically explore multiple combinations of hyperparameters using **KerasTuner’s Random Search** to identify the configuration that minimizes the **validation RMSE (Root Mean Squared Error)**.  
  - This step replaces manual trial-and-error with an efficient, systematic search across a defined hyperparameter space.

- **Initialize the tuner:**  
  - Use `kt.RandomSearch()` to automatically explore a defined number of random hyperparameter combinations.  
  - Each trial trains a model with a unique set of hyperparameters.  
  - The tuner tracks validation performance and records the configuration that minimizes the chosen metric (`val_rmse`).  
  - Key parameters:
    - **`objective`** → Metric to optimize (`val_rmse` for regression, minimized here).  
    - **`max_trials`** → Number of random combinations to test (start small, e.g., 5–10).  
    - **`directory` / `project_name`** → Location to save tuning logs and model checkpoints.  
    - **`seed`** → Ensures reproducible tuning results.

- **Define EarlyStopping:**  
  - The `keras.callbacks.EarlyStopping()` callback monitors validation performance and halts training when improvements stop.  
  - This prevents overfitting and saves computation time.  
  - Key arguments:
    - **`monitor='val_rmse'`** → Tracks validation RMSE.  
    - **`patience=5`** → Stops after 5 epochs with no improvement.  
    - **`mode='min'`** → Minimizes the metric (lower RMSE = better).  
    - **`restore_best_weights=True`** → Automatically reverts to the best model weights.

- **Run the tuning process:**  
  - Use `tuner.search()` to start the hyperparameter optimization.  
  - The tuner builds and trains multiple models using different hyperparameter sets.  
  - Each trial:
    - Trains the model for up to 100 epochs (usually fewer due to EarlyStopping).  
    - Reserves 20% of training data for validation (`validation_split=0.2`).  
  - Progress is printed to track each trial’s performance.

- **Retrieve the best hyperparameters:**  
  - Once tuning completes, call `tuner.get_best_hyperparameters(num_trials=1)` to extract the best configuration.  
  - Access it using `[0]` to get the top result and `.values` to view the hyperparameter dictionary.  
  - These optimal parameters can be used to rebuild and retrain the model for final evaluation.

**Summary:**  
This workflow automates hyperparameter optimization with **Random Search**, while **EarlyStopping** ensures efficient training and prevents overfitting.  
The result is a set of **best-performing hyperparameters** that minimize validation error and improve model generalization.


In [ ]:
# ============================================================
# STEP 1 — Initialize the Random Search Tuner
# ============================================================

# KerasTuner RandomSearch explores random combinations of hyperparameters
# to identify the model configuration that minimizes validation RMSE.
tuner = kt.RandomSearch(
    build_model,                               # model-building function
    objective=kt.Objective('val_rmse', 'min'), # metric to minimize (validation RMSE)
    max_trials=5,                              # number of random hyperparameter combinations to test
    seed=42,                                   # ensures reproducibility
    directory='kt_rand',                       # directory to store tuner results
    project_name='crop_yield'                  # subfolder name for this tuning experiment
)

# ============================================================
# STEP 2 — Define EarlyStopping Callback
# ============================================================

# EarlyStopping halts training automatically if validation RMSE stops improving,
# which prevents overfitting and reduces computation time.
early_stop = keras.callbacks.EarlyStopping(
    monitor='val_rmse',        # metric to monitor
    patience=5,                # stop after 5 epochs of no improvement
    mode='min',                # minimize RMSE
    restore_best_weights=True  # revert to the model weights with best validation RMSE
)

# ============================================================
# STEP 3 — Run the Random Search Tuning Process
# ============================================================

# The tuner trains multiple models, each with a unique combination of hyperparameters.
# Each trial uses a validation split to evaluate performance and rank results.
tuner.search(
    X_train, y_train,
    epochs=100,                 # maximum epochs per trial; EarlyStopping typically halts earlier
    validation_split=0.2,       # use 20% of training data for validation
    callbacks=[early_stop],     # apply EarlyStopping during training
    verbose=2                   # print progress for each trial
)

# ============================================================
# STEP 4 — Retrieve the Best Hyperparameter Combination
# ============================================================

# Once tuning completes, extract the best hyperparameters based on validation RMSE.
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

# Display the optimal hyperparameter configuration
print("Best hyperparameters:", best_hps.values)


### **1.5.5 Retrain and Evaluate the Best Model**

💡 **Tips**

- **Purpose:**  
  Rebuild and train the model using the **best hyperparameters (`best_hps`)** found during tuning.  
  Evaluate its final performance on the **held-out test set** to confirm how well the tuned model generalizes to unseen data.

- **Build and Train:**  
  - `tuner.hypermodel.build(best_hps)` reconstructs the model architecture using the optimal hyperparameter configuration.  
  - Train the model on `(X_train, y_train)` using the same settings as in tuning:  
    - **`validation_split=0.2`** → reserves 20% of training data for validation.  
    - **`callbacks=[early_stop]`** → applies EarlyStopping to prevent overfitting.  
  - Keeping these parameters consistent ensures comparable training behavior and performance.

- **Predictions & Evaluation:**  
  - Use `best_model.predict(X_test)` to generate predictions on unseen data.  
  - Compute:  
    - **RMSE** → `mean_squared_error(y_test, y_pred, squared=False)` for average prediction error.  
    - **R²** → `r2_score(y_test, y_pred)` for the proportion of explained variance.  
  - Lower RMSE and higher R² indicate better predictive accuracy and generalization.

- **Visualization:**  
  - Plot **Observed vs Predicted** values to assess model fit.  
  - Perfect predictions align along the **red dashed 1:1 line**.  
  - Deviations from the line highlight bias or variance patterns.


In [ ]:
# ============================================================
# STEP 1 — Build the Best Model from Tuned Hyperparameters
# ============================================================
best_model = tuner.hypermodel.build(best_hps)

# ============================================================
# STEP 2 — Train the Model with Consistent Settings
# ============================================================
history = best_model.fit(
    X_train, y_train,
    epochs=100,                 # upper limit; EarlyStopping will likely stop earlier
    validation_split=0.2,       # consistent with tuning
    callbacks=[early_stop],     # reuse EarlyStopping for stable training
    verbose=2
)

# ============================================================
# STEP 3 — Evaluate Model Performance on Test Set
# ============================================================
y_pred_tuned_tf = best_model.predict(X_test).ravel()  # flatten predictions for metrics

rmse_best = mean_squared_error(y_test, y_pred_tuned_tf, squared=False)
r2_best   = r2_score(y_test, y_pred_tuned_tf)

print("\nModel Performance on Test Set (with hyperparameter tuning):")
print(f"RMSE : {rmse_best:.4f}")
print(f"R²   : {r2_best:.4f}")

In [ ]:
# Visualize Observed vs Predicted
plt.figure(figsize=(5,4))
plt.scatter(y_test, y_pred_tuned_tf, alpha=0.6, edgecolor='none')
lims = [min(y_test.min(), y_pred_tuned_tf.min()), max(y_test.max(), y_pred_tuned_tf.max())]
plt.plot(lims, lims, 'r--', linewidth=1.2, label="Perfect prediction (1:1 line)")
plt.xlabel("Observed ln(yield)", fontsize=14)
plt.ylabel("Predicted ln(yield)", fontsize=14)
plt.title("Observed vs Predicted (Tuned Neural Network using TensorFlow)", fontsize=14)
plt.legend()
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()


## **1.6 Using the Trained Model to Predict Yield Changes under +1 °C Warming**

This section applies the trained neural network model to estimate how county-level crop yields respond to a **+1 °C increase in mean temperature**, relative to the **2012–2017 average climate conditions**.  
For detailed background on this climate-impact methodology, refer to *Assignment 3*.

### **1.6.1 Generate County-Level Predicted Yield (2012–2017 Average)**

💡 **Tips**

- **Objective:**  
  Estimate each county’s representative crop yield under average climate and management conditions from **2012–2017** using the trained model.

- **Key Steps:**
  1. **Aggregate climatology:**  
     Collapse multiple years into county-level means to represent long-term average conditions:  
     ```python
     county_means = df.groupby(["GEOID", "STATEFP"])[features_withoutSTATEFP].mean().reset_index()
     ```
  2. **Preserve FIPS codes:**  
     Convert `GEOID` and `STATEFP` to strings to retain leading zeros required for spatial joins.
  3. **Prepare features:**  
     Align columns to match the model’s input feature list and one-hot encode `STATEFP` for model compatibility.  
  4. **Predict and back-transform:**  
     - Model outputs are in **log-yield (ln_yield)**.  
     - Apply `np.exp()` to convert predictions to original yield units.  
  5. **Validate results:**  
     Confirm that predicted yields fall within plausible crop-specific ranges.

In [ ]:
# --- Compute county-level mean climate features (2012–2017 average) ---
features_withoutSTATEFP = ["sm", "precip", "tmean", "vpdmean"]

county_means = (
    df.groupby(["GEOID", "STATEFP"])[features_withoutSTATEFP]
      .mean()
      .reset_index()
)

# --- Prepare model input features ---
X_county_means = county_means[features]

# One-hot encode STATEFP for model compatibility
X_county_means = pd.get_dummies(X_county_means, columns=["STATEFP"])

# --- Predict county-level yields ---
county_means["pred_ln_yield"] = best_model.predict(X_county_means)
county_means["pred_yield"] = np.exp(county_means["pred_ln_yield"])  # back-transform

county_means.head()

### **1.6.2 Generate Predicted County Yield under +1 °C Scenario**

💡 **Tips**

- **Objective:**  
  Simulate how a **+1 °C increase in mean temperature (`tmean`)** affects county-level crop yields, assuming all other climate and management variables remain constant.  
  This sensitivity experiment quantifies the potential impact of uniform warming on crop productivity.

- **Key Steps:**
  1. **Create the +1 °C scenario:**  
     - Copy the baseline dataset (`X_county_means`) to `X_county_plus1C`.  
     - Increase the `tmean` variable by **+1 °C** to represent the warming scenario.  

  2. **Predict yields under the warming scenario:**  
     - Use the trained model to generate **log-transformed yield predictions** (`pred_ln_yield_plus1C`).  
     - Apply `np.exp()` to convert predictions back to the original yield units.  

  3. **Compute yield change metrics:**  
     - **Absolute change:**  
       $
       \Delta Yield = Yield_{+1°C} - Yield_{Baseline}
       $  
     - **Percentage change:**  
       $
       \%\Delta Yield = \frac{\Delta Yield}{Yield_{Baseline}} \times 100
       $

  4. **Interpretation:**  
     - **Negative values** → yield decline under warming (heat stress).  
     - **Positive values** → yield increase (possible benefit in cooler regions).  

  5. **Output:**  
     - Store all results in the same DataFrame (`county_means`) for subsequent spatial mapping and visualization.

In [ ]:
# --- Create +1°C scenario ---
X_county_plus1C = X_county_means.copy()
X_county_plus1C["tmean"] = X_county_plus1C["tmean"] + 1  # add +1°C to mean temperature

# --- Predict yields under the +1°C scenario ---
county_means["pred_ln_yield_plus1C"] = best_model.predict(X_county_plus1C)
county_means["pred_yield_plus1C"] = np.exp(county_means["pred_ln_yield_plus1C"])  # back-transform to yield units

# --- Compute yield differences (absolute and percentage) ---
county_means["pred_yield_plus1C_dif"] = (
    county_means["pred_yield_plus1C"] - county_means["pred_yield"]
)
county_means["pred_yield_plus1C_dif_percent"] = (
    county_means["pred_yield_plus1C_dif"] / county_means["pred_yield"] * 100
)

# Preview results
county_means.head()

### **1.6.3 Merge Geospatial Shapefile with Tabular Data**

💡 **Tips**

- **Objective:**  
  Join county yield predictions with county geometries to enable mapping and spatial analysis.

- **Key steps:**  
  1. Ensure `GEOID` and `STATEFP` are strings in both `county_means` and shapefiles.  
  2. Read county (`tl_2023_us_county.shp`) and state (`tl_2023_us_state.shp`) shapefiles.  
  3. Align coordinate reference systems (CRS).  
  4. Merge shapefile and tabular data using `GeoDataFrame.merge(on=["GEOID","STATEFP"])`.  
  5. Keep only states present in your dataset for cleaner maps.  
  6. Check merge results (`.isna().sum()`, row counts, CRS consistency).

In [ ]:
# --- Filepaths ---
url_counties = "tl_2023_us_county/tl_2023_us_county.shp"
url_states = "tl_2023_us_state/tl_2023_us_state.shp"

# --- Read shapefiles ---
counties = gpd.read_file(url_counties)
states = gpd.read_file(url_states)

# --- Align CRS ---
states = states.to_crs(counties.crs)

# --- Merge yield data with county geometries ---
counties_merged = counties.merge(county_means, on=["GEOID", "STATEFP"])

# --- Keep only states that have data ---
state_fips_with_data = counties_merged["STATEFP"].unique()
states_with_data = states[states["STATEFP"].isin(state_fips_with_data)]

### **1.6.4 Visualization — Predicted Yield Maps**

💡 **Tips**

- **Objective:**  
  Visualize county-level predicted yields under three scenarios:
  - Baseline (2012–2017 average)  
  - +1 °C warming scenario  
  - Yield change (+1 °C – baseline)

- **Map design:**  
  - **Colormaps:**  
    - Sequential (“YlGn”) → for baseline & +1 °C maps.  
    - Diverging (“BrBG”, “RdBu”, “coolwarm”) → for yield change map.  
  - **Color limits:**  
    Keep consistent `vmin/vmax` across maps for fair visual comparison.  
  - **Styling:**  
    - County borders: `edgecolor='grey', linewidth=0.2`  
    - State borders: black, thicker lines for clarity.  
    - Remove axes (`ax.axis("off")`) and add descriptive titles.  
  - **Interpretation:**  
    - Brown/red → yield decline (heat stress).  
    - Green/blue → yield increase or resilience.  

- **Output:**  
  Save high-resolution maps with legends and titles for consistent presentation.

In [ ]:
# Yield Difference Map (+1°C – Baseline)

fig, ax = plt.subplots(1, 1, figsize=(12, 8))

counties_merged.plot(
    column="pred_yield_plus1C_dif",
    cmap="BrBG",
    linewidth=0.2,
    ax=ax,
    edgecolor="grey",
    legend=True,
    legend_kwds={'label': "Yield change (bushels/acre)"},
    vmin = -40,
    vmax = 40,
)

# overlay state borders with thicker lines
states_with_data.boundary.plot(ax=ax, color="black", linewidth=2)

ax.set_title("County-level Predicted Yield Change (+1°C minus baseline)", fontsize=14)
ax.axis("off")
plt.savefig("yield_change_map.png", dpi=300, bbox_inches="tight")
plt.show()

## **Part 2 — Classification: Recognizing Handwritten Digits (MNIST)**

This section demonstrates a step-by-step workflow for **image classification** using the **MNIST dataset**, a classic benchmark for computer vision tasks.  
You will learn how to preprocess image data, build a fully connected (Dense) neural network, and improve model performance through hyperparameter tuning with **KerasTuner**.

**Workflow Overview**
1. **Load and inspect** the MNIST dataset.  
2. **Visualize** sample images and check class balance.  
3. **Preprocess** the data for a dense neural network (flatten and normalize).  
4. **Build, train, and evaluate** a baseline dense model.  
5. **Tune hyperparameters** (layer size, activation, learning rate) using KerasTuner and retrain the best model.

**Dataset Overview**
- **Name:** MNIST (Modified National Institute of Standards and Technology).  
- **Type:** 70,000 grayscale images of handwritten digits (0–9).  
- **Dimensions:** Each image is **28 × 28 pixels**.  
- **Labels:** Integer digits from 0 to 9 (10 classes).  
- **Split:**  
  - 60,000 training samples  
  - 10,000 testing samples

## **2.1 Read the Data**

💡 **Tips**

- **Core Function:**  
  Load the MNIST dataset and inspect its structure before preprocessing or modeling.  
  This ensures that images and labels are correctly loaded, with the expected dimensions and value ranges.

- **Key Checks:**
  1. Confirm dataset dimensions (`.shape`) for both training and testing sets.  
  2. Examine the **pixel value range** (should be 0–255 for grayscale images).  
  3. Verify that **all 10 digit classes (0–9)** are present in the labels.  

- **Useful Functions:**  
  - `tf.keras.datasets.mnist.load_data()` → loads pre-split training and test sets.  
  - `.shape` → displays the number of samples and image dimensions.  
  - `.min()` / `.max()` → checks pixel intensity range.  
  - `np.unique()` → lists unique label classes to verify class completeness.

In [ ]:
# Load MNIST dataset from TensorFlow
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()

# Inspect dataset structure
print("Training images shape:", X_train.shape, "labels shape:", y_train.shape)
print("Test images shape:    ", X_test.shape,  "labels shape:", y_test.shape)
print("Pixel value range: min =", X_train.min(), "max =", X_train.max())
print("Unique class labels:", np.unique(y_train))

## **2.2 Visualize Samples and Class Distribution**

💡 **Tips**

- **Core Function:**  
  This step helps you **understand and validate the dataset** before training any model.  
  Visualization ensures that the image data and labels are correctly paired, classes are balanced, and no anomalies exist.

  Specifically, this section:
  - Confirms the **image–label alignment** (each image matches its digit).  
  - Checks **class balance** — verifying all digits (0–9) appear in similar proportions.  
  - Builds early **intuition** about how digits are written and how clean the dataset is.

- **Key Code Functions:**
  - `plt.imshow()` → displays grayscale images from the MNIST dataset.  
  - `plt.subplot()` → arranges multiple images in a grid layout.  
  - `plt.hist()` → plots a histogram showing how many samples exist for each class.  
  - `np.arange(11) - 0.5` → centers histogram bins on integer class labels (0–9).  
  - `plt.tight_layout()` and `plt.suptitle()` → improve figure spacing and readability.

In [ ]:
# Function to visualize a grid of sample images with labels
def show_samples(images, labels, n_rows=5, n_cols=5, title="MNIST samples"):
    plt.figure(figsize=(n_cols * 1.5, n_rows * 1.5))
    plt.suptitle(title, fontsize=16)
    for i in range(n_rows * n_cols):
        ax = plt.subplot(n_rows, n_cols, i + 1)
        plt.imshow(images[i], cmap="gray")
        plt.title(f"Label: {labels[i]}")
        plt.axis("off")
    plt.tight_layout(rect=[0, 0, 1, 0.95])
    plt.show()

# Display 25 example training images
show_samples(X_train, y_train, n_rows=5, n_cols=5, title="First 25 Training Images")

# Visualize the number of samples per digit class
plt.figure(figsize=(8, 4))
plt.hist(y_train, bins=np.arange(11) - 0.5, rwidth=0.8, color="skyblue", edgecolor="black")
plt.xticks(range(10))
plt.xlabel("Digit Label", fontsize=12)
plt.ylabel("Count", fontsize=12)
plt.title("Training Set Class Distribution", fontsize=14)
plt.grid(axis="y", alpha=0.3)
plt.show()

## **2.3 Build a Base Model**

💡 **Tips**

- **Core Function:**  
  Prepare and train a **fully connected (Dense)** neural network on flattened MNIST images to create a **baseline classifier**.  
  This model establishes a reference performance before applying hyperparameter tuning or more advanced architectures (e.g., convolutional neural networks).

- **Key Steps:**

  1. **Preprocess the data:**
     - Flatten each 2D image (28×28 pixels) into a **1D vector** of 784 features.  
     - Normalize pixel values from **[0–255] → [0–1]** for stable training and faster convergence.  
       ```python
       X_train_flat = X_train.reshape((-1, 28*28)).astype("float32") / 255.0
       ```

  2. **Build a simple Dense network:**
     - **Input:** 784 features (flattened pixels).  
     - **Hidden Layer:** 16 neurons, ReLU activation.  
     - **Output Layer:** 10 neurons (Softmax activation for classes 0–9).  
       ```python
       model = models.Sequential([
           layers.Dense(16, activation='relu', input_shape=(784,)),
           layers.Dense(10, activation='softmax')
       ])
       ```

  3. **Compile the model:**
     - **Optimizer:** `'adam'` — an adaptive learning rate algorithm that combines the benefits of RMSprop and momentum for stable, efficient convergence.  
     - **Loss:** `'sparse_categorical_crossentropy'` — used when class labels are **integers (not one-hot encoded)**.  
       - Unlike `'categorical_crossentropy'`, which expects labels in one-hot encoded form (e.g., `[0,0,0,1,0,0,0,0,0,0]`),  
         `'sparse_categorical_crossentropy'` allows labels in simple integer form (e.g., `3` for the digit “3”).  
       - This simplifies preprocessing and saves memory, especially for large datasets.  
     - **Metric:** `'accuracy'` — measures the proportion of correctly classified images.

  4. **Train the model:**
     - Use **`batch_size=32`** to process 32 samples per gradient update (a common, balanced choice).  
     - Reserve **10% of the training data** for validation (`validation_split=0.1`).  
     - Add **EarlyStopping** to stop training automatically when validation loss stops improving:  
       ```python
       early_stop = EarlyStopping(
           monitor='val_loss',
           patience=3,
           restore_best_weights=True
       )
       ```

  5. **Evaluate model performance:**
     - Compute accuracy on the test set to gauge baseline performance.  
     - Typical accuracy for this small network is **~94–95%** on MNIST.

  6. **Visualize predictions:**
     - Display the first 15 test images with their true and predicted labels.  
     - **Green titles** → correct predictions; **Red titles** → incorrect ones.


In [ ]:
# ============================================================
# STEP 1 — Preprocess for Dense Neural Network
# - Flatten 28x28 images → 784-d vectors
# - Normalize pixel values from [0,255] to [0,1]
# ============================================================
X_train_flat = X_train.reshape((-1, 28 * 28)).astype("float32") / 255.0
X_test_flat  = X_test.reshape((-1, 28 * 28)).astype("float32") / 255.0

print("After flattening: X_train_flat shape =", X_train_flat.shape)


# ============================================================
# STEP 2 — Build a Simple Dense (Fully Connected) Model
# ============================================================
model = models.Sequential([
    layers.Dense(16, activation='relu', input_shape=(784,)),  # hidden layer
    layers.Dense(10, activation='softmax')                    # output layer (10 digits)
])


# ============================================================
# STEP 3 — Compile the Model
# ============================================================
model.compile(
    optimizer='adam',                             # adaptive optimizer
    loss='sparse_categorical_crossentropy',       # suitable for integer labels
    metrics=['accuracy']                          # track classification accuracy
)

# Display architecture summary
model.summary()


# ============================================================
# STEP 4 — Train the Model (with EarlyStopping)
# ============================================================
from tensorflow.keras.callbacks import EarlyStopping

early_stop = EarlyStopping(
    monitor='val_loss',          # monitor validation loss
    patience=3,                  # stop if no improvement after 3 epochs
    restore_best_weights=True,   # revert to best weights
    verbose=1
)

history = model.fit(
    X_train_flat, y_train,
    epochs=50,                   # generous limit; EarlyStopping will stop earlier
    batch_size=32,               # process 32 samples per update
    validation_split=0.1,        # hold out 10% of training data for validation
    callbacks=[early_stop],      # enable early stopping
    verbose=2
)


# ============================================================
# STEP 5 — Evaluate Model Performance on the Test Set
# ============================================================
test_loss, test_acc = model.evaluate(X_test_flat, y_test, verbose=0)
print(f"\nTest accuracy: {test_acc:.4f}")

In [ ]:
# ============================================================
# STEP 6 — Visualize Predictions (first 15 test samples)
# - Green title = correct prediction; Red = incorrect
# ============================================================

# Generate predicted class probabilities for the first 15 test samples
pred_probs = model.predict(X_test_flat[:15])

# Convert probabilities (10 per sample) to the most likely class label (0–9)
pred_labels = np.argmax(pred_probs, axis=1)

# Create a visualization grid for comparison
plt.figure(figsize=(12, 4))
for i in range(15):
    ax = plt.subplot(3, 5, i + 1)
    plt.imshow(X_test[i], cmap="gray")

    # Color code predictions: green = correct, red = incorrect
    color = "green" if pred_labels[i] == y_test[i] else "red"
    plt.title(f"True: {y_test[i]}  Pred: {pred_labels[i]}", color=color, fontsize=10)
    plt.axis("off")

# Add a descriptive title for the figure
plt.suptitle("Model Predictions on First 15 Test Images", fontsize=14, weight="bold")
plt.tight_layout(rect=[0, 0, 1, 0.95])
plt.show()

## **2.4 Hyperparameter Tuning**
**Goal:**  
  Build a tunable model structure that KerasTuner can repeatedly adjust and train using different hyperparameter combinations.  
  The tuner evaluates each configuration based on **validation accuracy** and reports the best-performing setup.


### **2.4.1 Define the Model Builder Function for Classification**

💡 **Key Idea:**  
The `build_model(hp)` function defines how the neural network is constructed using tunable parameters.  
KerasTuner will repeatedly call this function during the search, substituting different hyperparameter values to find the optimal setup.

- **Tunable parameters include:**
  - `n_layers`: number of hidden layers to use (1–3).  
  - `units`: number of neurons per hidden layer (32–256).  
  - `activation`: activation function (`'relu'`, `'tanh'`, `'elu'`).  
  - `learning_rate`: optimizer learning rate (`1e-2`, `1e-3`, `1e-4`).  

- **Core function workflow:**
  1. Define an input layer for flattened 28×28 images.  
  2. Dynamically add the number of Dense layers specified by `n_layers`, each with tuned `units` and `activation`.  
  3. Add a 10-neuron output layer with **softmax** activation for multi-class classification.  
  4. Compile the model with the **Adam** optimizer and the selected `learning_rate`.


In [ ]:
def build_model(hp):
    model = keras.Sequential()
    model.add(layers.Input(shape=(784,)))  # input layer for flattened 28x28 images

    # Tune number of hidden layers (1–3)
    n_layers = hp.Int('n_layers', min_value=1, max_value=3)
    for i in range(n_layers):
        # Tune number of units and activation function for each layer
        units = hp.Int(f'units_{i}', min_value=32, max_value=256, step=32)
        activation = hp.Choice(f'activation_{i}', ['relu', 'tanh', 'elu'])
        model.add(layers.Dense(units, activation=activation))

    # Output layer (10 neurons for 10 digit classes)
    model.add(layers.Dense(10, activation='softmax'))

    # Tunable learning rate for Adam optimizer
    lr = hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=lr),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    return model

### **2.4.2 Initialize the Tuner (Random Search)**

💡 **Tips**

- **Core Function:**  
  Set up a **KerasTuner RandomSearch** tuner to automatically test multiple combinations of hyperparameters defined in the `build_model(hp)` function.  
  The tuner will evaluate each model configuration based on **validation accuracy**, rank the results, and store the best-performing models.

- **Key Parameters:**
  - **`build_model`** → function defining the tunable model structure.  
  - **`objective='val_accuracy'`** → metric to maximize during tuning (classification objective).  
  - **`max_trials=5`** → number of different hyperparameter combinations to test.  
  - **`executions_per_trial=1`** → number of times each configuration is trained (higher values average out randomness but increase runtime).  
  - **`directory` / `project_name`** → specify where KerasTuner will store search logs and results.  
  - **`seed`** → ensures reproducibility of tuning results.


In [ ]:
# ============================================================
# Initialize the Random Search Tuner
# ============================================================
tuner = kt.RandomSearch(
    build_model,                     # model-building function
    objective='val_accuracy',        # maximize validation accuracy
    max_trials=5,                    # number of random combinations to test
    executions_per_trial=1,          # train each configuration once
    directory='kt_random_class',     # folder to store tuner results
    project_name='mnist_class', # project name for this tuning task
    seed=42                          # random seed for reproducibility
)

### **2.4.3 Run the Tuner Search**

💡 **Tips**

- **Core Function:**  
  Execute the **hyperparameter search** using the KerasTuner **RandomSearch** tuner.  
  The tuner will repeatedly build, train, and evaluate models using different hyperparameter combinations defined in the `build_model(hp)` function.

- **Key Concepts:**
  - **EarlyStopping:**  
    Prevents overfitting and saves time by stopping training early when the validation accuracy stops improving for several epochs.  
    The best-performing model weights (based on validation accuracy) are automatically restored.
  
  - **Validation Split:**  
    `validation_split=0.2` reserves 20% of the training data for validation during tuning — essential for the tuner to evaluate each configuration objectively.
  
  - **Epochs:**  
    Set to 20 to give each trial enough training time. EarlyStopping will terminate training earlier if no improvement is observed.

- **Key Parameters:**
  - `monitor='val_accuracy'` → tracks validation accuracy as the metric to optimize.  
  - `patience=5` → waits 5 epochs before stopping if no improvement.  
  - `restore_best_weights=True` → reloads the model weights from the best epoch.


In [ ]:
# ============================================================
# Run the Tuner Search
# ============================================================

# Define EarlyStopping to prevent overfitting during tuning
early_stop = keras.callbacks.EarlyStopping(
    monitor='val_accuracy',       # metric to monitor
    patience=5,                   # stop if no improvement after 5 epochs
    mode='max',                   # maximize validation accuracy
    restore_best_weights=True     # restore best weights after stopping
)

# Execute hyperparameter search
tuner.search(
    X_train_flat, y_train,
    epochs=20,                    # upper limit; EarlyStopping may stop sooner
    validation_split=0.2,         # reserve 20% for validation
    callbacks=[early_stop],       # apply early stopping
    verbose=2                     # print detailed tuning progress
)


### **2.4.4 Retrieve and Train the Best Model**

💡 **Tips**

- **Core Function:**  
  Once the tuner completes the search, retrieve the **best hyperparameter configuration** found during tuning, rebuild the corresponding model, and train it more thoroughly to ensure optimal performance.


- **Key Steps:**
  1. **Retrieve best hyperparameters:**  
     Use `tuner.get_best_hyperparameters(num_trials=1)[0]` to access the top-performing hyperparameter set.
  2. **Inspect results:**  
     Print the best hyperparameters to understand the optimal architecture chosen by KerasTuner.
  3. **Rebuild the model:**  
     Recreate the neural network using `tuner.hypermodel.build(best_hps)` with the tuned parameters.
  4. **Train the final model:**  
     Train the model again (e.g., for 30 epochs) using the same `validation_split` and `EarlyStopping` callback to ensure stable, high-accuracy performance.


In [ ]:
# ============================================================
# Retrieve and Train the Best Model
# ============================================================

# Retrieve the best hyperparameters from the tuning search
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
print("\nBest hyperparameters found:")
for k, v in best_hps.values.items():
    print(f"  {k}: {v}")

# Rebuild the model using the best hyperparameters
best_model = tuner.hypermodel.build(best_hps)

# Retrain the best model with EarlyStopping for stability
history = best_model.fit(
    X_train_flat, y_train,
    epochs=30,                     # higher limit; EarlyStopping will halt early if needed
    validation_split=0.2,          # 20% of training data used for validation
    callbacks=[early_stop],        # stop when validation accuracy stops improving
    verbose=2
)

### **2.4.5 Evaluate and Visualize**

💡 **Tips**

- **Core Function:**  
  Evaluate the tuned model’s performance on unseen test data and visualize sample predictions to verify how accurately the model recognizes handwritten digits.

- **Key Steps:**

  1. **Evaluate model performance:**  
     - Use the test dataset (`X_test_flat`, `y_test`) to compute the final **accuracy** and **loss**.  
     - These metrics show how well the tuned model generalizes beyond the training data.
     ```python
     test_loss, test_acc = best_model.evaluate(X_test_flat, y_test, verbose=0)
     ```

  2. **Generate predictions:**  
     - The model outputs **class probabilities** (confidence scores for each digit) when calling `predict()`.  
       ```python
       pred_probs = best_model.predict(X_test_flat[:15])
       ```
       Each row in `pred_probs` is a vector of 10 values (for digits 0–9) representing how confident the model is in each class.  
     - To extract the most likely class for each sample, use:
       ```python
       pred_labels = np.argmax(pred_probs, axis=1)
       ```
       This picks the index (0–9) of the **maximum probability** in each row — giving the final **predicted digit label**.

  3. **Visualize results:**  
     - Display a few test images with their **True (T)** and **Predicted (P)** labels.  
     - Use color cues to make results clear:  
       - 🟩 **Green title:** correct prediction.  
       - 🟥 **Red title:** incorrect prediction.  
     - This visual check helps confirm that the model correctly identifies digit patterns and highlights any common misclassifications.


In [ ]:
# ============================================================
# Evaluate and Visualize Model Performance
# ============================================================

# Evaluate tuned model on the test dataset
test_loss, test_acc = best_model.evaluate(X_test_flat, y_test, verbose=0)
print(f"\nTest accuracy: {test_acc:.4f}")

# Generate predictions for the first 15 test samples
pred_probs = best_model.predict(X_test_flat[:15])
pred_labels = np.argmax(pred_probs, axis=1)

# Visualize true vs predicted labels
plt.figure(figsize=(10, 4))
for i in range(15):
    ax = plt.subplot(3, 5, i + 1)
    plt.imshow(X_test[i].reshape(28, 28), cmap="gray")
    color = "green" if pred_labels[i] == y_test[i] else "red"
    plt.title(f"T:{y_test[i]}  P:{pred_labels[i]}", color=color, fontsize=9)
    plt.axis("off")

plt.suptitle("First 15 Test Images: True (T) vs Predicted (P)", fontsize=14)
plt.tight_layout(rect=[0, 0, 1, 0.95])
plt.show()
plt.show()

# **Assignment 4: Neural Network Models**

---

### **Submission Instructions**
- Begin with a **new Jupyter Notebook** containing your code, figures, and explanations **clearly separated by each question**.  
- Save and submit your notebook using the filename format:  
  **Lastname_Firstname_NetID_Assignment4.html**  
- Label all figures and tables with **informative titles, axis labels, and legends**.  
- Provide concise interpretations of your results and ensure all outputs are reproducible.

---

## **Part 1. Regression for Soybean Yield**  (4 pts)

### **1.1 Scikit-learn Neural Network Regression (MLPRegressor)**  (2 pts)

**Objective**  
Develop both a baseline and tuned **feed-forward neural network** using scikit-learn’s `MLPRegressor` to predict county-level soybean yields.  
Evaluate how hyperparameter optimization affects model accuracy, stability, and generalization.

**Tasks**  
1. Load the dataset `US_soyb.csv` into a pandas DataFrame and inspect its structure.  
2. Define predictor features (`X`) and the target variable (`y = ln_yield`).  
3. Apply **one-hot encoding** to categorical features (e.g., `STATEFP`) to make the dataset numeric.  
4. Split the data into **training (70%)** and **testing (30%)** subsets using `train_test_split()`.  
5. Build and evaluate a **baseline model** using default parameters of `MLPRegressor`.  
6. Use **RandomizedSearchCV** or **GridSearchCV** to tune the following hyperparameters:  
   - `hidden_layer_sizes`, `activation`, `alpha`, and `learning_rate_init`.  
7. Report:  
   - The **best parameters**, **cross-validation RMSE**, and **R²** score.  
8. Evaluate the tuned model on the test set and compare it with the baseline model.  

---

### **1.2 TensorFlow/Keras Neural Network Regression (KerasTuner)**  (2 pts)

**Objective**  
Use TensorFlow’s **Keras API** and **KerasTuner** to build, tune, and evaluate a neural network regression model for soybean yield prediction.  
Explore how different architectures, activation functions, and learning rates affect predictive performance.

**Tasks**  
1. Prepare the dataset as in Section 1.1 (normalized features, train–test split).  
2. Define a tunable **Sequential model** with:  
   - Three fully connected (**Dense**) hidden layers,  
   - Tunable hyperparameters for the number of neurons, activation functions, and learning rate.  
3. Initialize a **KerasTuner RandomSearch** tuner and define an appropriate search space.  
4. Implement an **EarlyStopping callback** that halts training when validation RMSE fails to improve for **10 consecutive epochs**, restoring the best weights.  
5. Run the tuner search to identify the **best hyperparameter configuration** (lowest validation RMSE).  
6. Rebuild and retrain the model using the best hyperparameters.  
7. Evaluate the tuned model on the test set and report:  
   - **RMSE** and **R²** metrics,  
   - An **Observed vs. Predicted** scatter plot (include a 1:1 reference line).  
8. Summarize tuning improvements and discuss how to prevent **underfitting** or **overfitting**.  
9. **Warming Scenario Analysis:**  
   - Simulate a **+1°C mean temperature** scenario and predict yield responses.  
   - Visualize the spatial distribution of yield changes.  
   - Compare the spatial patterns of the results with your **tree-based models** (e.g., Random Forest, XGBoost) in Assignment 3 and interpret key similarities or differences.

---

## **Part 2. Classification for Handwritten Images (Fashion MNIST)**  (4 pts)

### **2.1 Baseline Dense Neural Network**  (2 pts)

**Objective**  
Train a simple **dense neural network** using the **Fashion MNIST** dataset to classify images into ten clothing categories.  
Establish baseline accuracy before tuning.

**Tasks**  
1. Load the **Fashion MNIST dataset** using `tf.keras.datasets.fashion_mnist.load_data()`.  
2. Display the image dimensions (28 × 28 pixels).  
3. Visualize a grid of sample images with their corresponding labels.  
4. Plot the class distribution to confirm all categories are well represented.  
5. Preprocess the data:  
   - Flatten each 28×28 image into a **784-dimensional vector**,  
   - Normalize pixel values to the **[0, 1] range** by dividing by 255.  
6. Build a **Sequential model** with:  
   - One hidden layer (choose the number of neurons and activation),  
   - One output layer (Softmax activation for multi-class classification).  
7. Compile the model using:  
   - Optimizer: `adam`,  
   - Loss: `sparse_categorical_crossentropy`,  
   - Metric: `accuracy`.  
8. Train the model with a **validation split of 0.1**, `batch_size=32`, and multiple epochs.  
9. Report both **training** and **test accuracy**.  
10. Visualize several test images, displaying **True (T)** vs **Predicted (P)** labels, color-coded (green = correct, red = incorrect).

---

### **2.2 Tuned Classification Model with KerasTuner**  (2 pts)

**Objective**  
Use **KerasTuner** to systematically optimize hyperparameters for the Fashion MNIST classification model and evaluate performance improvements over the baseline.

**Tasks**  
1. Define a tunable model function `build_model(hp)` that allows tuning of:  
   - Number of hidden layers (1–3),  
   - Neurons per layer (32–256),  
   - Activation functions (`'relu'`, `'tanh'`, `'elu'`),  
   - Learning rate (`1e-2`, `1e-3`, `1e-4`).  
2. Initialize a **RandomSearch tuner** to maximize validation accuracy (`val_accuracy`).  
3. Add an **EarlyStopping callback** to stop training when validation accuracy does not improve for **5 epochs**.  
4. Run the tuner search and record the **best hyperparameter combination**.  
5. Rebuild and retrain the model using the optimal hyperparameters.  
6. Evaluate the tuned model on the test set and report:  
   - **Accuracy** and **Loss**,  
   - Performance comparison with the baseline model.  
7. Visualize 15 test images showing **True (T)** vs **Predicted (P)** labels (green = correct, red = incorrect).  
8. Discuss how the tuned model differs from the baseline.

---

## **Part 3. Summary of Insights**  (2 pt)

**Objective**  
Reflect on your experience training neural network models for both the **regression (soybean yield)** and **classification (Fashion MNIST)** tasks.  
Summarize key takeaways about model architecture, data handling, and performance across different problem types.

**Tasks**  
1. Compare how neural networks address **regression** and **classification** tasks, focusing on differences in architecture design, loss function, and evaluation metrics.  
2. Discuss how the **nature of the data** (tabular vs. image) affects model structure, preprocessing requirements, and learning complexity.  
3. Highlight the main challenges encountered during training (e.g., convergence, overfitting) and how tuning or early stopping helped mitigate them.  